# nutné knihovny:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# aktivační funkce

In [2]:
def sigmoid(z):
    return np.where(z<0, np.exp(z)/(1+np.exp(z)), 1/(1+np.exp(-z)))

In [3]:
def sigmoid_der(x):
    return np.vectorize(lambda y: y*(1-y))(x)

# ztrata

In [4]:
def ztratova_fce(e):
    return sum(e**2)

In [5]:
def ztratova_der(e):
    return -1 * (2*e)

# sit:

In [6]:
class sit:
    def __init__(self, x, y, p_neuronu, act_fce, der_act, loss_fce, der_loss):
        self.x = x
        self.y = y
        self.pn = p_neuronu
        self.ni = np.shape(x)[0]
        self.nj = np.shape(x)[1]
        self.act_fce = act_fce
        self.der_act = der_act
        self.loss_fce = loss_fce
        self.der_loss = der_loss
        # generovani nahodnych vychzich koeficientu:
        self.vahy1 = (np.random.rand(self.nj, self.pn)+0.5)/self.nj
        self.vahy2 = (np.random.rand(self.pn,1)+0.5)/self.pn
    
        self.forward_prop()
        
    def forward_prop(self):
        self.LO1 = self.act_fce(np.matmul(self.x,self.vahy1))
        self.y_hat = self.act_fce(np.matmul(self.LO1,self.vahy2))
        self.e = self.y - self.y_hat
    
    def backward_prop(self):
        self.delta1 = np.matmul(np.transpose(self.x),
                               np.matmul((self.der_loss(self.e) * self.der_act(self.y_hat)),
                                        np.transpose(self.vahy2))* self.der_act(self.LO1))
        self.delta2 = np.matmul(np.transpose(self.LO1),(self.der_loss(self.e)*self.der_act(self.y_hat)))
        
        self.vahy1 = self.vahy1 - self.delta1 * self.LR
        self.vahy2 = self.vahy2 - self.delta2 * self.LR
    
    def forecast(self,X_new):
        self.Lf = self.act_fce(np.matmul(X_new,self.vahy1))
        self.predict = self.act_fce(np.matmul(self.Lf,self.vahy2))
        return self.predict
        
    def train(self, n_epochs, LR):
        self.LR = LR
        self.n_epochs = n_epochs
        
        for i in range(self.n_epochs):
            self.backward_prop()
            self.forward_prop()
        
        

In [7]:
class minibatch:
    def __init__(self, x, y, p_neuronu, act_fce, der_act, loss_fce, der_loss,n_batch):
        self.x = x
        self.y = y
        
        self.pn = p_neuronu
        self.ni = np.shape(x)[0]
        self.nj = np.shape(x)[1]
        
        self.act_fce = act_fce
        self.der_act = der_act
        self.loss_fce = loss_fce
        self.der_loss = der_loss
        
        self.n_batch = n_batch
        self.data_id = np.array([np.random.choice(range(n_batch)) for i in range(self.ni)])
        
        self.site = []
        
        self.total_sit = sit(x, y, p_neuronu, act_fce, der_act, loss_fce, der_loss)
        
        for i in range(n_batch):
            self.site.append(sit(x[self.data_id == (i),:], y[self.data_id == (i),:], 
                                 p_neuronu, act_fce, der_act, loss_fce, der_loss))
        
        self.mean_vahy()
        
    def mean_vahy(self):
        self.mean_vahy1 = np.zeros((self.nj, self.pn))
        self.mean_vahy2 = np.zeros((self.pn,1))
        for i in range(self.n_batch):
            self.mean_vahy1 += (self.site[i].vahy1 / self.n_batch)
            self.mean_vahy2 += (self.site[i].vahy2 / self.n_batch)
    
    def pass_mean(self):
        for i in range(self.n_batch):
            self.site[i].vahy1 = self.mean_vahy1
            self.site[i].vahy2 = self.mean_vahy2
            self.site[i].forward_prop()
        self.total_sit.vahy1 = self.mean_vahy1
        self.total_sit.vahy2 = self.mean_vahy2
        self.total_sit.forward_prop()
        
    def train(self, n_epochs, LR):
        self.LR = LR
        for j in range(n_epochs):
            for i in range(self.n_batch):
                self.site[i].train(1,self.LR)
            self.mean_vahy()
            self.pass_mean()
            if j % 100 ==0:
                print(f"{j} iterace, ztrata: {self.loss_fce(self.total_sit.e)}")
        
        
            

# Dating

In [8]:
data = pd.read_csv("dating(1).csv")

In [9]:
data

,age,age_o,samerace,importance_same_race,importance_same_religion,pref_o_attractive,pref_o_sincere,pref_o_intelligence,pref_o_funny,pref_o_ambitious,...,intelligence_partner,funny_partner,ambition_partner,shared_interests_partner,interests_correlate,like,guess_prob_liked,met,decision,match
0,21,23,0,2,4,30.0,5.0,15.0,40.0,5.0,...,8.0,7.0,6.0,8.0,0.61,7.0,6.0,0,1,1
1,21,24,0,2,4,30.0,10.0,20.0,10.0,10.0,...,7.0,7.0,6.0,6.0,0.21,6.0,6.0,0,1,1
2,21,28,0,2,4,50.0,0.0,25.0,10.0,0.0,...,8.0,9.0,8.0,8.0,0.28,7.0,7.0,0,1,1
3,24,23,1,2,5,30.0,5.0,15.0,40.0,5.0,...,7.0,6.0,5.0,7.0,-0.21,7.0,8.0,0,1,1
4,24,28,1,2,5,50.0,0.0,25.0,10.0,0.0,...,8.0,9.0,4.0,5.0,0.11,5.0,6.0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981,26,33,0,6,1,20.0,10.0,20.0,15.0,15.0,...,8.0,2.0,5.0,2.0,0.20,5.0,2.0,0,0,0
1982,22,30,0,6,2,14.0,18.0,16.0,18.0,18.0,...,10.0,8.0,8.0,4.0,-0.16,5.0,3.0,0,0,0
1983,30,22,0,2,2,17.0,10.0,18.0,20.0,25.0,...,6.0,6.0,4.0,5.0,-0.28,7.0,6.0,0,1,0
1984,21,23,1,8,7,25.0,5.0,30.0,15.0,5.0,...,10.0,10.0,10.0,10.0,-0.13,10.0,10.0,0,1,0


In [10]:
X = np.array(data.loc[:,  data.columns != "match"])
Y= np.array(data["match"])

In [11]:
Xs = np.apply_along_axis(lambda x: (x-min(x))/(max(x)-min(x)),0,X)
Ys = np.apply_along_axis(lambda x: (x-min(x))/(max(x)-min(x)),0,Y)

In [12]:
Ys.shape

(1986,)

In [13]:
Ys.resize(len(Ys),1)

In [14]:
Ys.shape

(1986, 1)

In [15]:
len(Ys)*0.2

397.20000000000005

In [16]:
# 397 hodnot pudje do test a zbytek do train!

In [17]:
# zamichani a vyber :)
s = np.random.choice(len(Ys),size=(len(Ys)),replace=False)

In [18]:
Xs_test = Xs[s[0:396],:]
Xs_train = Xs[s[397:len(Ys)],:]
Ys_test = Ys[s[0:396],:]
Ys_train = Ys[s[397:len(Ys)],:]

In [19]:
np.shape(Xs_test)

(396, 39)

In [20]:
dating = minibatch(Xs_train,Ys_train,10, sigmoid, sigmoid_der, ztratova_fce, ztratova_der,50)

In [21]:
dating.train(1000,0.1)

0 iterace, ztrata: [401.27378386]
100 iterace, ztrata: [201.84165956]
200 iterace, ztrata: [151.96079311]
300 iterace, ztrata: [138.79355686]
400 iterace, ztrata: [132.32137075]
500 iterace, ztrata: [128.38590162]
600 iterace, ztrata: [125.77102702]
700 iterace, ztrata: [123.94458046]
800 iterace, ztrata: [122.61364291]
900 iterace, ztrata: [121.60676799]


In [24]:
np.sum(np.round(dating.total_sit.y_hat) == dating.y) /len(dating.y)
#  % dat je správně zatříděných!!!! (docela dobrej výsledek :) tj mam 90.37% uspesnost po 1000 iteracích! 

0.9018250471994965

In [25]:
# out of sample test - % spravne zatrídených:
fit=dating.total_sit.forecast(Xs_test)

sum(np.round(fit)==Ys_test) / len(Ys_test)

array([0.88383838])

In [26]:
# v testu na testovaci mnozine dosahuji 87% spolehlivosti! :)

# titanic:

In [27]:
data = pd.read_csv("titanic.csv")

In [28]:
data

,pclass,survived,sex,age
0,1,1,1,29.0000
1,1,1,0,0.9167
2,1,0,1,2.0000
3,1,0,0,30.0000
4,1,0,1,25.0000
...,...,...,...,...
1041,3,0,0,45.5000
1042,3,0,1,14.5000
1043,3,0,0,26.5000
1044,3,0,0,27.0000


In [29]:
X = np.array(data.loc[:,  data.columns != "survived"])
Y= np.array(data["survived"])

In [30]:
Xs = np.apply_along_axis(lambda x: (x-min(x))/(max(x)-min(x)),0,X)
Ys = np.apply_along_axis(lambda x: (x-min(x))/(max(x)-min(x)),0,Y)

In [31]:
Ys.resize(len(Ys),1)

In [32]:
s = np.random.choice(len(Ys),size=(len(Ys)),replace=False)

Xs_test = Xs[s[0:199],:]
Xs_train = Xs[s[200:len(Ys)],:]
Ys_test = Ys[s[0:199],:]
Ys_train = Ys[s[200:len(Ys)],:]

In [33]:
titanic = minibatch(Xs_train,Ys_train,3, sigmoid, sigmoid_der, ztratova_fce, ztratova_der,50)

In [34]:
titanic.train(1000,0.1)

0 iterace, ztrata: [233.53582895]
100 iterace, ztrata: [200.06919341]
200 iterace, ztrata: [143.99019163]
300 iterace, ztrata: [130.34295268]
400 iterace, ztrata: [127.47133522]
500 iterace, ztrata: [125.89871799]
600 iterace, ztrata: [124.82300265]
700 iterace, ztrata: [124.04998329]
800 iterace, ztrata: [123.49232041]
900 iterace, ztrata: [123.08777445]


In [35]:
fit=titanic.total_sit.forecast(Xs_test)

In [36]:
# out of sample test - % spravne zatrídených:
sum(np.round(fit)==Ys_test) / len(Ys_test)

array([0.8040201])